<a href="https://colab.research.google.com/github/akamojo/PAD-project/blob/master/part2_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --force https://github.com/chengs/tqdm/archive/colab.zip

     | 768kB 1.2MB/s
  Stored in directory: /tmp/pip-ephem-wheel-cache-u5_uhcsq/wheels/41/18/ee/d5dd158441b27965855b1bbae03fa2d8a91fe645c01b419896
Successfully built tqdm
  Found existing installation: tqdm 4.28.1
    Uninstalling tqdm-4.28.1:
      Successfully uninstalled tqdm-4.28.1


In [0]:
import re
from tqdm import tqdm_notebook as tqdm

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
file = open("/content/drive/My Drive/STUDIA/SEM 8/pad/en1.8m.txt", "r", encoding="utf8")
# file = open("en1.8m.txt", "r", encoding="utf8")

whole_text = file.read()
text = whole_text.split('\n')

In [0]:
def append_space(text, pattern, left=True):
    p = re.compile("[" + pattern + "]")
    
    for i, m in enumerate(p.finditer(text)):
        if left:
            text = text[: m.start() + i] + " " + text[m.start() + i :]
        else:
            text = text[: m.start() + 1 + i] + " " + text[m.start() + 1 + i :]
            
    return text

In [0]:
def preprocessing(text):
    for i in range(len(text)):
        text[i] = append_space(text[i], ",\])>")
        text[i] = append_space(text[i], ".")
        text[i] = append_space(text[i], "\[(<", False)
      
    return text

In [0]:
def build_n_gram(text_arr, n_gram, grams):
    cur_gram = []
    count_gram = 0

    for i in range(len(text_arr)):

        cur_gram = [text_arr[i]]

        for j in range(i + 1, i + n_gram):
            if j < len(text_arr):
                cur_gram.append(text_arr[j])

        if len(cur_gram) == n_gram:
            try:
                grams[" ".join(cur_gram)] += 1
            except KeyError:
                grams[" ".join(cur_gram)] = 1

    return grams

In [0]:
def build_dict(text, n_s):
    dictionary = []
    for n in range(n_s):
        dictionary.append(dict())
    
    for n in range(n_s):
        for line in text:
            dictionary[n] = build_n_gram(line.split(), n + 1, dictionary[n])
        
    return dictionary

In [0]:
def SCP_f(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] ** 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] * dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [0]:
def dice(words, dictionary):
    words_arr = words.split()
    
    numerator = dictionary[len(words_arr) - 1][words] * 2
    F = 1 / (len(words_arr) - 1)
    denominator = 0
    
    for i in range(len(words_arr) - 1):
        left = words_arr[0:i+1]
        right = words_arr[i+1:]
        
        denominator += dictionary[len(left) - 1][" ".join(left)] + dictionary[len(right) - 1][" ".join(right)]
        
    F *= denominator
    return numerator / F

In [0]:
def LocalMax(text, table):
    xys = dict()
    
#     for nr in tqdm(range(len(text))):
    for nr in range(len(text)):
        line = text[nr]

        text_arr = line.split()

        if len(text_arr) < 2:
            continue

        for i in range(len(text_arr) - 2):
            cur_gram = " ".join(text_arr[i:i+2])

            j = i + 3
            n_gram = 1

            while j < len(text_arr) and n_gram < 6:
                next_gram = " ".join(text_arr[i:j])

                try:
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]

                cur_gram = next_gram

                j += 1
                n_gram += 1

            cur_gram = " ".join(text_arr[i:i+2])
            j = i - 1
            n_gram = 1

            while j >= 0 and n_gram < 6:
                next_gram = " ".join(text_arr[j:i+2])


                try:                        
                    xys[cur_gram][1] = max(xys[cur_gram][1], table[next_gram])
                    
                except KeyError:
                    if len(cur_gram.split()) > 2:
                        x = table[" ".join(cur_gram.split()[:-1])]
                        x = max(x, table[" ".join(cur_gram.split()[1:])])
                        xys[cur_gram] = [x, table[next_gram]]
                    else:
                        xys[cur_gram] = [-1, table[next_gram]]


                cur_gram = next_gram

                j -= 1
                n_gram += 1
              
    return xys

In [0]:
def main(text, max_gram, threshold, glue_fun, verbose=False):
    length = 0
    for line in text:
        length += len(line.split())
    
    if verbose:
        print("Preprocessing text")
    text = preprocessing(text)
    
    if verbose:
        print("Building dictionary with frequencies")

    dictionary = build_dict(text, max_gram)
    
    table = dict()
    
    if verbose:
        print("Proceeding to calculating glue value")
        print("Glue value calculated for:")

    for i in range(2, max_gram + 1):
        if verbose:
            print(i, "grams")
            
        for s in dictionary[i - 1]:
            table[s] = glue_fun(s, dictionary)
    
    if verbose:
        print("Proceeding to Localmax")
    
    RE = dict()
    xys = LocalMax(text, table)
            
    for g in xys:
        if xys[g][0] == -1:
            xys[g][0] = xys[g][1]
        
        val = (xys[g][0] + xys[g][1]) / 2
        
        if table[g] > val and re.match(r"^[a-z\'A-Z\s]*$", g) and dictionary[len(g.split()) - 1][g] > threshold:
            RE[g] = dictionary[len(g.split()) - 1][g] / (length / len(g.split()))
    
    return RE, dictionary

In [14]:
test = text[0]

RE, dictionary = main([test], 7, 1, SCP_f)
print("Obtained", len(RE), "Relevant Expressions")
RE

Obtained 2 Relevant Expressions


{'John Philoponus': 0.0380952380952381, 'in the': 0.0761904761904762}

In [0]:
def split_docs(whole_text):
    docs1 = whole_text.split('</doc')
#     docs = []
#     for d in docs1:
#         docs.append(d.split('<doc'))
        
#     return [item for sublist in docs for item in sublist]
    return docs1

In [17]:
import math

docs = split_docs(whole_text)
n = 1000
REs = []
word_count = dict()

for i in tqdm(range(n)):
    RE, dictionary = main(docs[i].split('\n'), 7, 1, SCP_f)
    REs.append(RE)
    
    for grams in dictionary:
        for g in grams:
            try:
                word_count[g] += 1
            except KeyError:
                word_count[g] = 1

/usr/local/lib/python3.6/dist-packages/tqdm/_tqdm_notebook.py:88: TqdmExperimentalWarning: Detect Google Colab 0.0.1a2 and thus load dummy ipywidgets package. Note that UI is different from that in Jupyter. See https://github.com/tqdm/tqdm/pull/640
  " See https://github.com/tqdm/tqdm/pull/640".format(colab.__version__), TqdmExperimentalWarning)


In [0]:
ex_keywords = []

for i in range(len(REs)):
    ex_keywords.append(dict())
    for r in REs[i]:
        tf = REs[i][r]
        idf = math.log10(n / float(word_count[r]))
        ex_keywords[-1][r] = tf * idf

In [28]:
print("--------------------------------------------------")
print("TOP 15 Explicit Keywords for documents")
print("--------------------------------------------------\n")

for i, l in enumerate(ex_keywords[:10]):
    print("Doc", i+1)
    print("--------------------------------------------------")

    for elem, val in sorted(l.items(), key=lambda kv: -kv[1])[:15]:
        print(elem, ":\t", val)
    print('\n')

--------------------------------------------------
TOP 15 Explicit Keywords for documents
--------------------------------------------------

Doc 1
--------------------------------------------------
may be given to any :	 0.00577478344562079
depart for the Holy Land :	 0.00577478344562079
removal of a hydrogen ion :	 0.00577478344562079
Indic scripts :	 0.0046198267564966315
head of a monastery :	 0.0046198267564966315
depart for the Holy :	 0.0046198267564966315
below the national average :	 0.0046198267564966315
removal of a hydrogen :	 0.0046198267564966315
hydrogen ion :	 0.0046198267564966315
a monastery :	 0.004156257947004456
may be given :	 0.0034648700673724736
murder of Naboth :	 0.0034648700673724736
Peter of Courtenay :	 0.0034648700673724736
below the national :	 0.0034648700673724736
Louis the Child :	 0.0034648700673724736


Doc 2
--------------------------------------------------
by a cannonball :	 0.021201413427561835
an independent record :	 0.021201413427561835
footb

In [0]:
word_doc = dict()

for i in range(len(REs)):
    for w in list(REs[i].keys()):
        try:
            word_doc[w].add(i)
        except KeyError:
            word_doc[w] = set()
            word_doc[w].add(i)

In [0]:
im_keywords = dict()

for w1 in word_doc:
    for w2 in word_doc:
        if w1 != w2:
            try:
                im_keywords[w1].append([w2, len(set.intersection(word_doc[w1], word_doc[w2]))])
            except KeyError:
                im_keywords[w1] = []
                im_keywords[w1].append([w2, len(set.intersection(word_doc[w1], word_doc[w2]))])
            
            try:
                im_keywords[w2].append([w1, len(set.intersection(word_doc[w1], word_doc[w2]))])
            except KeyError:
                im_keywords[w2] = []
                im_keywords[w2].append([w1, len(set.intersection(word_doc[w1], word_doc[w2]))])


In [54]:
print("--------------------------------------------------")
print("TOP Implicit Keywords for document 1")
print("--------------------------------------------------\n")

top0 = sorted(ex_keywords[0].items(), key=lambda kv: -kv[1])[:15]

# for k, v in top0:
#     print(k, v)
#     #print(im_keywords[k])
    
im_keywords[top0[0][0]]

# for i, l in enumerate(ex_keywords[:10]):
#     print("Doc", i+1)
#     print("--------------------------------------------------")

#     for elem, val in :
#         print(elem, ":\t", val)
#     print('\n')

--------------------------------------------------
TOP Implicit Keywords for document 1
--------------------------------------------------



[['in the', 1],
 ['of the', 1],
 ['John Philoponus', 1],
 ['body parts', 1],
 ['the end of', 1],
 ['civil war', 1],
 ['as well as', 1],
 ['Indic scripts', 1],
 ['Southeast Asia', 1],
 ['attached to', 1],
 ['a monastery', 1],
 ['Orthodox Church', 1],
 ['elevated to', 1],
 ['have been', 1],
 ['may be given', 1],
 ['in the', 1],
 ['of the', 1],
 ['John Philoponus', 1],
 ['body parts', 1],
 ['the end of', 1],
 ['civil war', 1],
 ['as well as', 1],
 ['Indic scripts', 1],
 ['Southeast Asia', 1],
 ['attached to', 1],
 ['a monastery', 1],
 ['Orthodox Church', 1],
 ['elevated to', 1],
 ['have been', 1],
 ['may be given', 1],
 ['given to any', 1],
 ['serve as', 1],
 ['head of a monastery', 1],
 ['World War', 1],
 ['World War II', 1],
 ['UEFA Cup', 1],
 ['United States', 1],
 ['His work', 1],
 ['the history of', 1],
 ['murder of Naboth', 1],
 ['adverse events', 1],
 ['He was', 1],
 ['his life', 1],
 ['New Orleans', 1],
 ['Pope Honorius', 1],
 ['depart for', 1],
 ['depart for the Holy', 1],
 ['dep